## Prueba 1 de rag con late chunking

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import openai
import nltk
from dotenv import load_dotenv
import os

#cargamos variables de entorno
load_dotenv()

# Descargamos los recursos de NLTK necesarios para el tokenizador de oraciones
nltk.download('punkt')

# Configura tu API key de OpenAI
openai.api_key = os.getenv('OPENAI_API_KEY')

# 1. Cargar el modelo de embeddings (puedes elegir otro que se adapte a tus necesidades)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Supongamos que tienes una lista de documentos (pueden ser textos largos de cada tema)
documents = [
    "Documento 1: Contenido extenso del tema A que es un tema muy especial y dificil.",
    "Documento 2: Contenido detallado del tema B ...",
    # Agrega más documentos según tu temario
]

# 3. Generar embeddings para cada documento
document_embeddings = embedding_model.encode(documents, convert_to_numpy=True)

# 4. Crear un índice FAISS para la búsqueda por similitud
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

# 5. Función de late chunking: fragmenta el documento solo cuando sea necesario
def late_chunking(text, max_chunk_tokens=300, overlap_tokens=50):
    """
    Fragmenta el texto en chunks de máximo max_chunk_tokens tokens, 
    agregando un solapamiento de overlap_tokens entre fragmentos.
    Se utiliza el tokenizador de oraciones de NLTK para dividir el texto.
    """
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = ""
    current_tokens = 0

    for sentence in sentences:
        sentence_tokens = sentence.split()
        num_tokens = len(sentence_tokens)
        
        # Si agregar esta oración excede el límite, crea un nuevo chunk
        if current_tokens + num_tokens > max_chunk_tokens:
            chunks.append(current_chunk.strip())
            # Para el solapamiento, tomar los últimos overlap_tokens del chunk anterior
            current_chunk_tokens = current_chunk.split()
            current_chunk = " ".join(current_chunk_tokens[-overlap_tokens:]) if overlap_tokens > 0 else ""
            current_tokens = len(current_chunk.split())
        
        current_chunk += " " + sentence
        current_tokens += num_tokens

    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# 6. Función para procesar la consulta del usuario
def answer_query(query, top_k=4):
    # Generar embedding para la consulta
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)
    
    # Buscar los top_k documentos más relevantes usando FAISS
    distances, indices = index.search(query_embedding, top_k)
    retrieved_docs = [documents[i] for i in indices[0]]
    
    # Aplicar late chunking en documentos que sean demasiado largos
    context_chunks = []
    for doc in retrieved_docs:
        if len(doc.split()) > 300:
            chunks = late_chunking(doc, max_chunk_tokens=800, overlap_tokens=150)
            context_chunks.extend(chunks)
        else:
            context_chunks.append(doc)
    
    # Combinar los chunks en un único contexto
    context = "\n\n".join(context_chunks)
    
    # Preparar el prompt para el modelo de lenguaje
    prompt = f"Contexto:\n{context}\n\nPregunta: {query}\nRespuesta:"
    
    # Usar la API de ChatCompletion con gpt-3.5-turbo
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Eres un asistente educativo experto. Solo has de responder en base a lo que tienes conocimiento."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,
        temperature=0.0,
    )
    
    return response.choices[0].message.content.strip()


# 7. Ejemplo de uso
if __name__ == "__main__":
    consulta = "Dime lo que sepas sobre el tema A"
    respuesta = answer_query(consulta)
    print("Respuesta del chatbot:")
    print(respuesta)


/Users/joseluis.fernandez/miniconda3/envs/dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joseluis.fernandez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Respuesta del chatbot:
El tema A es un tema muy especial y difícil, que requiere un profundo conocimiento y comprensión para poder abordarlo de manera adecuada. Puede involucrar conceptos complejos, teorías avanzadas o prácticas especializadas. Es importante dedicar tiempo y esfuerzo en estudiar y entender este tema, ya que puede ser fundamental en ciertos contextos académicos o profesionales. Si necesitas más información específica sobre el tema A, estaré encantado de ayudarte.
